# Kytchen tutorial

## 1. Ingredients

There is no cooking without ingredients, so let's see how we can handle them with Kytchen. First, we need to import the package.

In [1]:
import kytchen as ky

Ingredients are stored as objects of the `Ingredient` class. These objects are initialised with three arguments:

- The name of the ingredient.
- The calories of the ingredient per unit of measurement.
- The unit in which the ingredient is measured.

For example, let's say that we want to create an ingredient object for mineral water. We could do it as follows:


In [2]:
water = ky.Ingredient("Mineral water", 0, "L")

Here we have specified the name ("Mineral water"), the fact that it has zero calories per litre, and that we are going to measure it in litres (symbol "L"). Ingredient objects can be easily printed:

In [3]:
print(water)

Mineral water (0 kcal/L)


That is how ingredient objects can be created, but, if you want to use these objects in recipes, you will have to take a different route.

In order to store and use your ingredients in Kytchen, you must create a CSV file named `ingredients.csv`. Each row will represent an ingredient object and, within each row, the cells (from left to right) should represent the following:

1. A unique ID for the ingredient.
2. The full name of the ingredient.
3. The caloric content of the ingredient per unit of measurement.
4. The unit in which the ingredient is going to be measured.

If we wanted to store our water ingredient, our CSV file should include a row like this one:

Once we have a CSV file of this form, we can load all the ingredients in it using the `load_ingredients` function. This function is called automatically when Kytchen is started. In fact, you may have noticed that this tutorial already comes with a sample `ingredients.csv` file. Unless you have deleted it, the ingredients in it should have been loaded already.

Loaded ingredients can be accessed with the `find_component` function, which takes a single argument corresponding to the unique ID of the ingredient.

In [4]:
print(ky.find_component("slice-bread"))
print(ky.find_component("cheese"))

Slice of bread (70.0 kcal/u)
Cheese (5.0 kcal/g)


## 2. Recipes

We have ingredients. Now it's time for us to create recipes. In Kytchen, recipes are stored as objects of the `Recipe` class — not very original, huh? Objects of this class are initialised with the following arguments:

* **A dictionary of components.** The keys are the IDs of the ingredients used in the recipe and the values are the amounts that are needed (measured in the ingredients' respective units).
* **A list of steps.** Each step can be encoded with a string `step`. If you want to specify that the step takes `mm` minutes, the step can be encoded as the list `[step, mm]`. If you want to specify that it takes `mm` minutes and `ss` seconds, you can also encode it as `[step, [mm, ss]]`. The list of steps is optional and defaults to an empty array.
* A **name** for the recipe (optional, defaults to "Unnamed recipe").
* A string specifying the **date of creation** of the recipe (optional, defaults to "").

Let's see this in action creating a simple recipe.

In [5]:
components = {"slice-bread": 2, "cheese": 10}
steps = [
    "Go to the kitchen.", # Simple step.
    ["Prepare the sandwhich.", 2], # Step that takes two minutes.
    ["Celebrate.", [0, 30]]
    # ^^ Step that takes 30 seconds.
]

sandwich = ky.Recipe(components, steps, name = "Cheese sandwich", date = "1492-10-12")
print(sandwich)

Cheese sandwich (1492-10-12)
Servings: 1
Calories: 190 kcal

INGREDIENTS
2 u  Slice of bread
10 g  Cheese

METHOD
- Go to the kitchen.
- Prepare the sandwhich. +2:0
- Celebrate. +0:30



And that's how you create a recipe! Notice that Kytchen has automatically computed for us the calories of this meal.

If you want to scale a recipe to a certain number of servings (this number can be a float), this is all you have to do:

In [6]:
sandwich.print(servings = 2.3)

Cheese sandwich (1492-10-12)
Servings: 2.3
Calories: 437 kcal

INGREDIENTS
4.6 u  Slice of bread
23.0 g  Cheese

METHOD
- Go to the kitchen.
- Prepare the sandwhich. +2:0
- Celebrate. +0:30



And, thus, your ingredients are rescaled and the calories are recomputed accordingly.

We have seen that, when we print a recipe, the duration of its steps is displayed when it's defined. However, look at what happens if you create a recipe specifying the duration of all its steps.

In [7]:
steps = [
    ["Prepare the sandwhich.", 2],
    ["Celebrate.", [0, 30]]
]

sandwich = ky.Recipe(components, steps, name = "Cheese sandwich", date = "1492-10-12")
print(sandwich)

Cheese sandwich (1492-10-12)
Servings: 1
Calories: 190 kcal
Preparation time: 3 min

INGREDIENTS
2 u  Slice of bread
10 g  Cheese

METHOD
- Prepare the sandwhich. >2:0
- Celebrate. >2:30



In this case, we also get the total preparation time for the recipe (rounded to the nearest minute). Moreover, next to each step, we can find the exact time by which the step should be completed, counting from the start of the preparation of the recipe. This can be very convenient for time-tracking.

### Saving and loading recipes

Once you have created a recipe, you can save it to a JSON file using the `save` method and specifying a `path`. The recipe will be saved in `recipes/{path}.json`. The path must uniquely identify the recipe; in particular, it must be different from the ID of any ingredient. Once any recipe is saved in the `recipes` folder, it can always be accessed directly with `find_component` using its path. Let's see this in action.

In [8]:
sandwich.save("sandwich")
print(sandwich == ky.find_component("sandwich"))

True


And that's it! Now our recipe will always be accessible. If you reset Python and reload the package, `ky.find_component("sandwich")` will still give you the recipe.

The structure of the JSON files that store recipes is very simple; you can have a look at `recipes/sandwich.json` and see for yourself. Of course, in order to define a recipe, you can just create a JSON file with that structure and add it to the `recipes` folder.

### Using recipes in other recipes

Kytchen allows you to use saved recipes as ingredients for other recipes. Their "unit of measurement" is, of course, the number of servings.

Let's see an example.

In [9]:
components = {"sandwich": 2}
method = ["Put the two sandwiches on a plate.", "Enjoy."]

two_sandwiches = ky.Recipe(components, method, name = "Two sandwiches on a plate")
print(two_sandwiches)

two_sandwiches.save("two-sandwiches-plate") # Save it!

Two sandwiches on a plate
Servings: 1
Calories: 380 kcal

INGREDIENTS
2 serv  Cheese sandwich

METHOD
- Put the two sandwiches on a plate.
- Enjoy.



## 3. Meal plans

Kytchen has a `Mealplan` class for storing meal plans. Objects of this class are initialised with two arguments:
- A list with all the **meals that are going to be consumed**. Each item in this list must be a dictionary encoding the meals that will be consumed on a certain day: the keys must be the paths of the recipes and the values must be the corresponding numbers of servings (can be a float).
- A list with any **extra meals that we want to prepare**. Each item in the list must be a dictionary encoding the extra meals that should be prepared on a given day (maybe in anticipation for the meals on the following days). These dictionaries must have the same format as above.

As with recipes, there are also two optional arguments for the name and the creation date of meal plan objects.

Let's see how this works through an example. Say that we want to create a three-day meal plan with the following meals:
- Day 1: "Cheese sandwich" and "two sandwiches on a plate".
- Day 2: Two servings of "cheese sandwich".
- Day 3: "Two sandwiches on a plate".

However, we want to be efficient, so we want to prepare all the sandwiches on the first day. Therefore, we want to prepare four extra sandwiches on Day 1 (recall that "two sandwiches on a plate" includes two sandwiches).

This is how we can define this meal plan:

In [10]:
consume = [
    {"sandwich": 1, "two-sandwiches-plate": 1},
    {"sandwich": 2},
    {"two-sandwiches-plate": 1}
]

prepare = [
    {"sandwich": 4},
    {},
    {}
]

mp = ky.MealPlan(consume, prepare, name = "My meal plan", date = "Today!")
print(mp)

My meal plan (Today!)

Day 1
CONSUME:
- Cheese sandwich (servings: 1, 190 kcal)
- Two sandwiches on a plate (servings: 1, 380 kcal)
PREPARE:
- Two sandwiches on a plate (servings: 1)
- Cheese sandwich (servings: 7)

Day 2
CONSUME:
- Cheese sandwich (servings: 2, 380 kcal)
PREPARE:

Day 3
CONSUME:
- Two sandwiches on a plate (servings: 1, 380 kcal)
PREPARE:
- Two sandwiches on a plate (servings: 1)

Average daily energy: 444 kcal



There you have it! Kytchen tells us what we are consuming each day, what we have to prepare each day and the average daily caloric intake. Notice that, if a recipe uses another recipe as an ingredient (just as "two sandwiches on a plate" uses "sandwich"), the whole recipe is expanded, and we are asked to prepare the big recipe and all its component recipes separatedly. For example:
- In Day 1, we have to make seven sandwiches: four of these are for the next few days, one is for the sandwich on Day 1, and the last two are for the "two sandwiches on a plate" meal on Day 1, which is also displayed.
- In Day 3, we have no sandwiches to make (we already prepared them all on Day 1), but we need to use the sandwiches that we already have in order to prepare the "two sandwiches on a plate" recipe.

Now look at what would have happened if we had only made three extra sandwiches on the first day:

In [11]:
prepare = [{"sandwich": 3},{},{}]
mp = ky.MealPlan(consume, prepare)
print(mp)

Unnamed meal plan

Day 1
CONSUME:
- Cheese sandwich (servings: 1, 190 kcal)
- Two sandwiches on a plate (servings: 1, 380 kcal)
PREPARE:
- Two sandwiches on a plate (servings: 1)
- Cheese sandwich (servings: 6)

Day 2
CONSUME:
- Cheese sandwich (servings: 2, 380 kcal)
PREPARE:

Day 3
CONSUME:
- Two sandwiches on a plate (servings: 1, 380 kcal)
PREPARE:
- Two sandwiches on a plate (servings: 1)
- Cheese sandwich (servings: 1)

Average daily energy: 444 kcal



In this case, we would have to make the missing sandwich for "sandwiches on a plate" on Day 3.

And lastly, see what happens if we prepare too many sandwiches:

In [12]:
prepare = [{"sandwich": 2},{"two-sandwiches-plate": 3},{}]
mp = ky.MealPlan(consume, prepare)
print(mp)

Unnamed meal plan

Day 1
CONSUME:
- Cheese sandwich (servings: 1, 190 kcal)
- Two sandwiches on a plate (servings: 1, 380 kcal)
PREPARE:
- Two sandwiches on a plate (servings: 1)
- Cheese sandwich (servings: 5)

Day 2
CONSUME:
- Cheese sandwich (servings: 2, 380 kcal)
PREPARE:
- Cheese sandwich (servings: 6)
- Two sandwiches on a plate (servings: 3)

Day 3
CONSUME:
- Two sandwiches on a plate (servings: 1, 380 kcal)
PREPARE:

Average daily energy: 444 kcal

EXCEDENT:
2 Two sandwiches on a plate



In this case, we have an excedent, and Kytchen is able to identify it and let us know.

Once you have a meal plan, Kytchen can generate a shopping list for you. This shopping list specifies the ingredients and amounts needed to prepare all the meals contained in the meal plan.

In [13]:
print(mp.shopping_list())

Slice of bread: 10 u
Cheese: 50 g



Meal plans can be saved as JSON files with the `save` method, specifiying a `path`. The meal plan will be saved in `mealplans/{path}.json` and it will be added to the `mealplans` dictionary, with `path` as key. The function `load_mealplans` loads all the meal plans in the `mealplans` folder and adds them to the dictionary; this function is called automatically when Kytchen is started.

In [14]:
mp.save("mealplan")
print(ky.mealplans)

{'mealplan': Unnamed meal plan}
